In [93]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import math
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [94]:
%matplotlib inline

# Deal with data

## Import dataframe

In [95]:
df = pd.read_csv('fifa21_train.csv')
display(df.head())
df.shape

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


(11701, 101)

In [96]:
list(df.columns)

['ID',
 'Name',
 'Age',
 'Nationality',
 'Club',
 'BP',
 'Position',
 'Team & Contract',
 'Height',
 'Weight',
 'foot',
 'Growth',
 'Joined',
 'Loan Date End',
 'Value',
 'Wage',
 'Release Clause',
 'Contract',
 'Attacking',
 'Crossing',
 'Finishing',
 'Heading Accuracy',
 'Short Passing',
 'Volleys',
 'Skill',
 'Dribbling',
 'Curve',
 'FK Accuracy',
 'Long Passing',
 'Ball Control',
 'Movement',
 'Acceleration',
 'Sprint Speed',
 'Agility',
 'Reactions',
 'Balance',
 'Power',
 'Shot Power',
 'Jumping',
 'Stamina',
 'Strength',
 'Long Shots',
 'Mentality',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Defending',
 'Marking',
 'Standing Tackle',
 'Sliding Tackle',
 'Goalkeeping',
 'GK Diving',
 'GK Handling',
 'GK Kicking',
 'GK Positioning',
 'GK Reflexes',
 'Total Stats',
 'Base Stats',
 'W/F',
 'SM',
 'A/W',
 'D/W',
 'IR',
 'PAC',
 'SHO',
 'PAS',
 'DRI',
 'DEF',
 'PHY',
 'Hits',
 'LS',
 'ST',
 'RS',
 'LW',
 'LF',
 'CF',
 'RF',
 'RW',
 'LAM'

## Standardize Headers

In [97]:
def stHead(df):
    new_header = []
    for h in df.columns:
        x = h.lower().replace(' ','_')
        new_header.append(x)
    df.columns = new_header
    display(df)

In [98]:
stHead(df)

,id,name,age,nationality,club,bp,position,team_&_contract,height,weight,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,ova
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,B. Böðvarsson,25,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,...,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2,65
11697,214680,G. Gallon,27,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,...,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2,70
11698,221489,J. Flores,22,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,...,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2,67
11699,146717,Anderson Silva,26,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,...,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0,68


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11701 entries, 0 to 11700
Columns: 101 entries, id to ova
dtypes: float64(10), int64(43), object(48)
memory usage: 9.0+ MB


In [100]:
df.describe()

,id,age,growth,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,...,gk_reflexes,total_stats,base_stats,pac,sho,pas,dri,def,phy,ova
count,11701.000000,11701.00000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11660.000000,11701.000000,...,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000,11701.000000
mean,219436.649004,25.27049,5.534655,258.482181,51.593795,48.048116,53.494060,60.436800,45.067324,266.497222,...,15.768310,1629.786599,361.106316,68.095206,55.046406,58.893257,64.223229,50.030254,64.817964,66.921972
std,37480.006004,4.95764,5.810903,72.267012,17.872747,19.399617,16.956195,13.971811,17.732902,76.835287,...,17.151738,260.766470,40.318331,11.247282,13.851853,10.174939,9.818311,16.841283,9.757873,6.876081
min,27.000000,16.00000,-1.000000,33.000000,6.000000,3.000000,5.000000,8.000000,4.000000,43.000000,...,1.000000,731.000000,228.000000,26.000000,17.000000,25.000000,28.000000,12.000000,27.000000,38.000000
25%,204060.000000,21.00000,0.000000,232.000000,41.000000,33.000000,46.000000,56.000000,32.000000,232.000000,...,8.000000,1490.000000,333.000000,62.000000,46.000000,52.000000,59.000000,35.000000,59.000000,62.000000
50%,229040.000000,25.00000,4.000000,271.000000,56.000000,52.000000,56.000000,63.000000,47.000000,279.000000,...,11.000000,1659.000000,362.000000,69.000000,58.000000,59.000000,65.000000,53.000000,66.000000,67.000000
75%,244176.000000,29.00000,10.000000,306.000000,65.000000,64.000000,65.000000,69.000000,59.000000,319.000000,...,14.000000,1811.000000,389.000000,76.000000,65.000000,66.000000,71.000000,64.000000,72.000000,72.000000
max,259090.000000,43.00000,26.000000,437.000000,94.000000,95.000000,93.000000,94.000000,90.000000,470.000000,...,90.000000,2304.000000,490.000000,96.000000,93.000000,93.000000,95.000000,89.000000,93.000000,93.000000


## Identify numerical and categorical variables

In [101]:
list(df.dtypes)

[dtype('int64'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('float64'),
 dtype('float64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('float64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dtype('int64'),
 dt

In [102]:
df._get_numeric_data()

,id,age,growth,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,...,gk_reflexes,total_stats,base_stats,pac,sho,pas,dri,def,phy,ova
0,184383,26,1,258,54,47,43,70,44.0,286,...,6,1682,357,69,51,63,63,51,60,64
1,188044,30,0,365,66,79,76,68,76.0,375,...,16,1961,412,83,75,68,82,33,71,77
2,184431,33,0,336,73,76,34,78,75.0,424,...,3,1925,404,80,77,78,86,27,56,80
3,233796,22,13,242,44,42,58,62,36.0,259,...,12,1527,329,57,44,54,57,57,60,59
4,234799,23,8,249,49,37,61,68,34.0,280,...,15,1664,360,66,44,60,64,60,66,65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,232504,25,5,224,62,24,51,59,28.0,248,...,14,1589,347,75,28,56,59,60,69,65
11697,214680,27,4,84,12,14,13,29,16.0,96,...,72,1123,387,72,67,66,72,40,70,70
11698,221489,22,8,286,64,66,51,64,41.0,291,...,16,1603,337,76,60,60,72,26,43,67
11699,146717,26,7,254,64,66,51,73,NaN,284,...,24,1429,400,72,63,70,72,57,66,68


In [103]:
df.select_dtypes('object')

,name,nationality,club,bp,position,team_&_contract,height,weight,foot,joined,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,A. Pasche,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,Right,"Jul 1, 2015",...,59+1,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1
1,Alan Carvalho,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,Right,"Jan 16, 2015",...,53+2,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2
2,S. Giovinco,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,Right,"Jan 31, 2019",...,56+2,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2
3,J. Evans,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,Right,"Jul 1, 2016",...,58+2,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2
4,Y. Demoncy,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,Right,"Jul 1, 2018",...,64+2,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11696,B. Böðvarsson,Iceland,Jagiellonia Białystok,LB,LB,Jagiellonia Białystok 2018 ~ 2021,"6'1""",168lbs,Left,"Feb 8, 2018",...,60+2,60+2,60+2,63+2,63+2,61+2,61+2,61+2,63+2,16+2
11697,G. Gallon,France,ESTAC Troyes,GK,GK,ESTAC Troyes 2019 ~ 2022,"6'1""",174lbs,Right,"Jul 1, 2019",...,26+2,26+2,26+2,25+2,24+2,26+2,26+2,26+2,24+2,69+2
11698,J. Flores,Chile,CD Antofagasta,RM,LM CAM RM,CD Antofagasta 2019 ~ 2024,"5'6""",143lbs,Right,"Jan 23, 2019",...,44+2,44+2,44+2,49+2,45+2,35+2,35+2,35+2,45+2,17+2
11699,Anderson Silva,Brazil,Barnsley,CM,NaN,Barnsley 2010,"6'2""",179lbs,Right,NaN,...,68+0,68+0,68+0,66+0,64+0,60+0,60+0,60+0,64+0,25+0


In [104]:
print(df._get_numeric_data().columns)
print(df.select_dtypes('object').columns)

Index(['id', 'age', 'growth', 'attacking', 'crossing', 'finishing',
       'heading_accuracy', 'short_passing', 'volleys', 'skill', 'dribbling',
       'curve', 'fk_accuracy', 'long_passing', 'ball_control', 'movement',
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance',
       'power', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots',
       'mentality', 'aggression', 'interceptions', 'positioning', 'vision',
       'penalties', 'composure', 'defending', 'marking', 'standing_tackle',
       'sliding_tackle', 'goalkeeping', 'gk_diving', 'gk_handling',
       'gk_kicking', 'gk_positioning', 'gk_reflexes', 'total_stats',
       'base_stats', 'pac', 'sho', 'pas', 'dri', 'def', 'phy', 'ova'],
      dtype='object')
Index(['name', 'nationality', 'club', 'bp', 'position', 'team_&_contract',
       'height', 'weight', 'foot', 'joined', 'loan_date_end', 'value', 'wage',
       'release_clause', 'contract', 'w/f', 'sm', 'a/w', 'd/w', 'ir', 'hits',
       'ls', '

## Deal with NaN

In [105]:
#df.isna().sum()[df.isna().sum() != 0]
#nan_df = pd.Dataframe(df.isna().sum()[df.isna().sum() != 0]) #, c

In [106]:
nan_df = pd.DataFrame(df.isna().sum(), columns=['NaN'])

nanCols = list(nan_df[nan_df.NaN != 0].index)

#nanCols
nan_df.T[nanCols]

,club,position,joined,loan_date_end,volleys,curve,agility,balance,jumping,interceptions,positioning,vision,composure,sliding_tackle,a/w,d/w
NaN,18,288,41,11072,41,41,41,41,41,7,7,41,279,41,59,59


In [107]:
#nanCols = list(df.isna().sum()[df.isna().sum() != 0].index)
#nanCols

In [108]:
# Percentage of NaN

In [109]:
nan_df = pd.DataFrame(round(df[nanCols].isna().sum()/len(df),4)*100, columns=['percent_nan'])
nan_df.T

,club,position,joined,loan_date_end,volleys,curve,agility,balance,jumping,interceptions,positioning,vision,composure,sliding_tackle,a/w,d/w
percent_nan,0.15,2.46,0.35,94.62,0.35,0.35,0.35,0.35,0.35,0.06,0.06,0.35,2.38,0.35,0.5,0.5


In [110]:
# Drop loan_date_end variable
df.drop(columns='loan_date_end', inplace=True)
nanCols.remove('loan_date_end')

In [111]:
# Personal question: check if the 41 NaN values coincide in the same rows 
df.volleys[df.joined.isna()].value_counts()
#==df.volleys.isna().iloc[0]
# df.joined.isna()==df.volleys.isna()[df.joined.isna()==df.volleys.isna() == True]

Series([], Name: volleys, dtype: int64)

In [112]:
nan41cols = df.isna().sum()[df.isna().sum() == 41]
nan41cols

joined            41
volleys           41
curve             41
agility           41
balance           41
jumping           41
vision            41
sliding_tackle    41
dtype: int64

In [113]:
# Drop all rows with NaN values
dfc = df.dropna(axis=0)
dfc.shape

(11211, 100)

In [116]:
len(df)

11701

In [115]:
dfc.isna().sum()[dfc.isna().sum() != 0]

Series([], dtype: int64)

In [117]:
# We have dropped a 4.18% of rows